# SLIM Elastic Net Recommender Implementation

In [13]:
# Data import
from utils.data_util import load_URM, load_data, combine_matrices, create_submission, write_submission
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# Model
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt


# Evaluator
from Evaluation.Evaluator import EvaluatorHoldout

In [14]:
URM_all = load_URM()

interactions_and_impressions found!


## Split into training and test set
We split the data into 85% of training set and 15% of test set.

In [15]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage=0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1560 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 925 ( 2.2%) Users that have less than 1 test interactions


In [16]:
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs


hyperparameters_range_dictionary = {
    "topK": Integer(1000, 4000),
    "l1_ratio": Real(low = 1e-5, high = 0.1, prior = 'log-uniform'),
    "alpha": Real(low = 1e-5, high = 0.1, prior = 'uniform'),
}

output_folder_path = "../hyper_param_tuning/"

n_cases = 50  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},     # Additional hyperparameters for the fit function
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},     # Additional hyperparameters for the fit function
)

hyperparameterSearch = SearchBayesianSkopt(SLIMElasticNetRecommender,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "best",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = SLIMElasticNetRecommender.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 1180, 'l1_ratio': 4.466981343032434e-05, 'alpha': 0.07700407177401801}
SLIMElasticNetRecommender: Processed 5784 (23.6%) in 5.00 min. Items per second: 19.28
SLIMElasticNetRecommender: Processed 12075 (49.3%) in 10.00 min. Items per second: 20.12
SLIMElasticNetRecommender: Processed 19599 (80.0%) in 15.00 min. Items per second: 21.77
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 18.96 min. Items per second: 21.54
EvaluatorHoldout: Processed 40069 (100.0%) in 43.00 sec. Users per second: 932
SearchBayesianSkopt: New best config found. Config 0: {'topK': 1180, 'l1_ratio': 4.466981343032434e-05, 'alpha': 0.07700407177401801} - results: PRECISION: 0.0369438, PRECISION_RECALL_MIN_DEN: 0.0717616, RECALL: 0.0690270, MAP: 0.0163458, MAP_MIN_DEN: 0.0316115, MRR: 0.1232141, NDCG: 0.0633793, F1: 0.0481287, HIT_RATE: 0.2768225, ARHR_ALL_HITS: 0.1413498, NOVELTY: 0.0039215, AVERAGE_P

KeyboardInterrupt: 

In [ ]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path=output_folder_path)
search_metadata = data_loader.load_data(SLIMElasticNetRecommender.RECOMMENDER_NAME + "_metadata.zip")
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

In this notebook we are creating a TopPop recommender, we load it from the course repository.

In [ ]:
recommender = SLIMElasticNetRecommender(URM_train)
recommender.fit(l1_ratio=best_hyperparameters['l1_ratio'],alpha=best_hyperparameters['alpha'], topK=best_hyperparameters['topK']) # Train the model

## Evaluate the model

In [ ]:
evaluator_test = EvaluatorHoldout(URM_test, [10])
result_df, _ = evaluator_test.evaluateRecommender(recommender)
print(result_df.loc[10])

## Create and save the submission
We can now create the submission and save it as csv.

In [ ]:
submission = create_submission(recommender)
write_submission(submission=submission, file_name='SLIM_ElasticNet_HT_'+'['+str(result_df.loc[10]['MAP'])+']')